Script para fazer o jogo com o planejador central

In [1]:
using Distributed

In [2]:
nprocs = 2
addprocs(nprocs);

In [3]:
@everywhere using DataFrames, LinearAlgebra, Statistics

Funções do jogo

In [4]:
@everywhere function solve_tree(data, game_parameters, cf, grid=100, hlow=0.01, hhigh=1.0, grid_payment=1000)
    
#     #data ####
#     #all the data is escalated so as to D = 1.0
    Vmax = data[1]; #firm's maximum reorganization value
    L = data[2]; #firm's maximum liquidation value
    Dj = data[3]; #junior's debt face value
    Ds = 1.0 - Dj; #senior's debt face value
    D = Ds + Dj; #total debt, used to escale costs

#     #parameters####
#     #parameters include only the parameters necessary to calculate the model, so it does not include θs0, θj0, μ
    ρ = game_parameters[1]; # (1 - ρ) is the firm's depreciation rate each period
    β = game_parameters[2]; #inverse of the speed of learning
    c0 = game_parameters[3]; #fixed cost of going to court
    λj = game_parameters[4]; #chance of j being called to propose at each period

    #c1 is calibrated. We will start with the same c1 as Dou et al.(2019)
    c1 = c0/30; #variable cost for each period at court
    #Initial definitions: costs, maximum value of the firm, maximum number of periods####
    
    

    tsym = cf[1]
    ac = cf[2]
    acs = cf[3]
    acj = cf[4]

    #number of periods
    function max_turns(Vmax, L, ρ, c0, c1, ac)
        t = 0

        #using the specification of the theoretical model
        #we are not considering the change in indexation from t to t+1 yet

        #the condition L - c0 - c1*t >= 0 assures that the liquidation payoffs won't be negative

        while( (ρ^(t-1) * Vmax > L) && (L - c0 - c1*t - ac >= 0) )
            t = t+1
        end
        return t

    end


    T = max_turns(Vmax, L, ρ, c0, c1, ac)

    #cost function
    function Ct(t, c0=c0, c1=c1)
        #cost at period t=0(index1) is 0
        if(t <= 1)
            return 0
        else
            return c0 * D + c1 * (t-1) * D #test to make index==1 be t==0
        end
    end

    #maximum value of reorganization each period
    function Vt(Vmax, ρ, t, ac)

        if(t <=1)
            return Vmax
        else
            #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
            return ρ^(t-2) * Vmax - ac
        end


    end
    #index 1 corresponds to t=0 and 2 to t=1...

    #liquidation payoffs


    function p_L(t, ac)
    
        #central planner recovers all the liquidation payoff
        return min(L - Ct(t) - ac)
    
    end
        
    #grid size
    grid = grid

    hlow = hlow
    hhigh = hhigh


    hvals = LinRange(hlow, hhigh, grid)

    #array to tell the size of the pie at period t, according to reorganization skill levels
    #arguments: (period, θkt)
    U = zeros(T+1, grid)


    for t in 1:T+1
        for h in 1:grid
            U[t,h] = hvals[h] * Vt(Vmax, ρ, t, ac) - Ct(t)
        end
    end

    #cumulative distribution function
    function cdf(x, lt, β=β)

        #return the cdf of x given the lower bound lt
        #geq than 1 because of our discretization method 
        if(x >= 1.0)

            return 1.0

        else

            if(x >= lt)
                return 1.0 - ((1.0-x)^β)/((1-lt)^β)
            else
                return 0.0
            end
        end
    end


    #pmf: each point of the grid is at the center of the cdf
    function pmf_cdf(lt, δ = hlow/2, β=β)
        pmf = zeros(grid)

        for (i, h) in enumerate(hvals)
            pmf[i] = cdf(h+δ, lt) - cdf(h-δ, lt)
        end

        return pmf
    end



    #generating the pmfs

    #probability mass function
    pmf = zeros(grid,grid)

    δ = hlow/2
    for (i, htoday) in enumerate(hvals)
        for (j, htomorrow) in enumerate(hvals)
            pmf[i,j] = cdf(htomorrow+δ, htoday) - cdf(htomorrow-δ, htoday)
        end
    end
    


    #we need a pmf for when we have lkt and want to know θk,t+1
    #we will call it 'pmf2'
    pmf2 = zeros(grid,grid)

    for i in eachindex(hvals)
        for j in eachindex(hvals)
            pmf2[i, j] = sum(pmf[i, :] .* pmf[:, j])
        end
    end

    #continuation value
    #period t (not t+1), θst, θjt
    p_W = zeros(T+1, grid, grid);

    #optimal payments
    #period, θst, θjt, outputs
    Ppt_array = zeros(T, grid, grid, 5); #we won't need payment proposal for period T, a.k.a. "T+1"

    #populating the last period with the liquidation values
    p_W[T+1, :,:] .= p_L(T+1, ac);

# p_L(T+1, ac)

# U[T+1, 100]

    function dot_W(t, hst, hjt, grid, p_W, pmf)

        #calculates the dot product between k_W and pmf

        payoff=0.0
        @inbounds for s in hst:grid
            @inbounds for j in hjt:grid
                
                payoff+= p_W[t+1, s, j] * pmf[hst, s] * pmf[hjt, j]
            
            end
        end
        return payoff
    end

    #lkt+1 = hkt, so we input it here

    function dot_U(t, hkt, grid, U, pmf)

        #calculates the dot product between U and pmf
        payoff=0.0
        @inbounds for i in hkt:grid
            payoff+= (U[t, i]) * pmf[hkt, i]
        end
        return payoff
    end
    
    

# dot_U(T, 97, grid, U, pmf)
#ou seja, o threshold é 97



# dot_W(T, hst, hjt, grid, p_W, pmf)

    #function to calculate the optimal payment offer and directly populate the array

    function Ppt!(t, hst, hjt, p_W, p_L, Ppt_array, pmf, pmf2, U, Vmax=Vmax, grid=grid, acs=acs, acj=acj, ac=ac)
    
        #pmf of m is a function of t (at t=0 (index 1), the information is symmetric)
        #array: pmf[lmt, 1:end] or pmf2[lmt, 1:end]

        #payoffs matrix####
        hpt = max(hst, hjt)
        payoff_reorg = dot_U(t, hpt, grid, U, pmf)

        #calculating the optimal policy between liquidating, reorganizing or waiting ####
        payoff_liq = p_L(t, ac)

        #waiting payoff is associated with the first possible payment, the waiting offer(-Vmax)
        payoff_wait = dot_W(t, hst, hjt, grid, p_W, pmf) #não precisa de probm aqui porque não há adversário

        payoff_max, policy = findmax((payoff_liq, payoff_wait, payoff_reorg))

        #populates the array
        Ppt_array[t, hst, hjt, :] .= payoff_liq, payoff_wait, payoff_reorg, payoff_max, policy

    end

    #optimal proposal
    function propose(t, hst, hjt, Ppt_array)


        #it just searches for the corresponding Pkt
        payoff_max, policy = @views Ppt_array[t, hst, hjt, end-1:end]

        return payoff_max, policy
    end

    function populate_ppt_manual!(t, p_W, p_L, Ppt_array, pmf, pmf2, U, Vmax, grid, acs, acj, ac)

        @inbounds for s in 1:grid
            for j in 1:grid
                Ppt!(t, s, j, p_W, p_L, Ppt_array, pmf, pmf2, U, Vmax, grid, acs, acj, ac) #Ppt
            end
        end

    end
    
#     populate_ppt_manual!(T, p_W, p_L, Ppt_array, pmf, pmf2, U, Vmax, grid, acs, acj, ac)

#checando se é simétrico nas habilidades porque elas são "substitutas perfeitas"
# Ppt_array[T,100,10,:] == Ppt_array[T,10,100,:]

    function populate_wkt_manual!(t, p_W, p_L, Ppt_array, pmf, pmf2, U, Vmax, grid, acs, acj, ac)

        for s in 1:grid
            for j in 1:grid
                p_W[t, s, j] = propose(t, s, j, Ppt_array)[1]
            end
        end
    end

    function populate_periods_manual!(tfinal, tbegin, p_W, p_L, Ppt_array, pmf, pmf2, U, Vmax, grid, acs, acj, ac)

#         @time begin


            for t in tfinal:-1:tbegin

                #populating Pkt
                populate_ppt_manual!(t, p_W, p_L, Ppt_array, pmf, pmf2, U, Vmax, grid, acs, acj, ac)

                #populating Wkt
                populate_wkt_manual!(t, p_W, p_L, Ppt_array, pmf, pmf2, U, Vmax, grid, acs, acj, ac)

            end

#         end

    end

   ### Populating the whole game
    populate_periods_manual!(T, 1, p_W, p_L, Ppt_array, pmf, pmf2, U, Vmax, grid, acs, acj, ac)

    
    ### returns the arrays necessary to simulate the game
    return p_W, Ppt_array
    
end

Simulation code

In [5]:
function simulate_moments(U, S, N, game_parameters, simulation_parameters, cf, data1, data2, A1, A2, n1)


    Results = loop_simulations(U, S, N, game_parameters, simulation_parameters, cf, data1, data2, A1, A2, n1)

    Moments = zeros(S, 8)

    μ = simulation_parameters[3]
    for s in 1:S

        DF = DataFrame()
        DF.payoff = Results[s,:,1];
        DF.out = Results[s,:,3];
        DF.t = Results[s,:,4];
        DF.observed_proposals = Results[s,:,5];

        DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


        #desfazendo o deslocamento no índice de t
        DF.t = DF.t .- 1.0;

        # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

        INCOURT = filter(DF -> DF.t .> 0.0, DF);
        PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

        REORG = filter(DF -> DF.outcome .== "R", DF);#casos reorganizados dentro e fora da corte

        PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome .== "R", PRECOURT)
        INCOURT_REORG = filter(INCOURT -> INCOURT.outcome .== "R", INCOURT)

        #cálculo dos momentos
        #1. avg log number of months between observed proposals incourt

        mm1 = INCOURT
        mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
        mm1 = mean(mm1)

        #2. fraction reorganized given that the case went into court

        mm2 = INCOURT
        mm2 = size(filter(mm2 -> mm2.outcome .== "R", mm2),1)/ size(mm2,1)

        #3. ln duration of court cases in months

        mm3 = INCOURT
        mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
        mm3.t = mm3.t .* μ
        mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

        #4. fraction of cases incourt
        mm4 = size(INCOURT,1) / size(DF, 1)

        #5. avg recovery rate for S given a REORGANIZATION in the 25% fastest cases
        #moment is not calculated by the central planner
        mm5 = zero(0.0)
        

        #6. avg recovery rate for J given a REORGANIZATION in the 25% fastest cases
        #moment is not calculated by the central planner
        mm6 = zero(0.0)


        #7. junior avg fraction gain given incourt REORGANIZATION
        #moment is not calculated by the central planner

        mm7 = zero(0.0)
        #8. total recovery rate given incourt REORGANIZATION

        mm8 = INCOURT_REORG
        mm8 = mean( mm8.payoff)

        Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8]


        #if there are no INCOURT or no PRECOURT cases, it will return NaN
        #thus, we replace it by zero
        Moments[s,:] .= replace!(Moments[s,:], NaN=> zero(0.0))

    end





    #9. slope of the regression of log(recovery rate | incourt reorg) ~ log(duration)

    #fazendo a regressão do nono momento fora do loop para poupar tempo
    #isto é, ao invés de fazer uma regressão para cada simulação,
    #faremos uma regressão só usando todas as simulações
    df = DataFrame()

    for s in 1:S
        df_result = DataFrame(Results[s,:, 1:4])
        append!(df, df_result)
    end

    rename!(df, [:x1, :x2, :x3, :x4] .=> [:payoff, :payoff_j, :out, :t])



    #desfazendo o deslocamento no índice de t
    df.t = df.t .- 1.0;

    #keeping only incourt reorganizations
    df = filter(df -> df.t .> 0.0, df);
    df = filter(df -> df.out .== 3.0, df)


    x = log.(df.payoff)

    X = [ones(size(x,1)) x]

    y = log.(df.t .* μ)

    if(size(df, 1) == 0 || rank(X' * X) != size(X,2))
        #the second condition is to make sure that (X' * X) is singular before calculating mm9
        mm9 = zero(0.0)
    else
        mm9 = (X' * X) \ (X' * y)
        mm9 = mm9[2]
    end


    MM = vec(mean(Moments, dims=1))
    MM = vcat(MM, mm9)

    return MM

end



function loop_simulations(U, S, N, game_parameters, simulation_parameters, cf, data1, data2, A1, A2, n1)


    #U is an array if dimensions S x N x 120 of fixed draws of the uniform distribution
        #120 is the upper bound of draws needed: 4 for each round (with cram dowm), maximum of 30 rounds
    #S is the number of simulations
    #N is the number of observations

    #data1 is the center of the first cluster
    #A1 is the collection of arrays from the first cluster (s_W, j_W, Pst, Pjt)
    #n1 is the number of observations in the first cluster



    #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster
    Results = zeros(S, N, 5)

    #últimas entradas de Results são os valores das dívidas


    for s in 1:S
        for n in 1:N
            if(n <= n1)
                Results[s,n,1:5] .= simulate_game(U[s,n,:], 1, data1, game_parameters, simulation_parameters, cf, A1[1], A1[2])
            else
                Results[s,n,1:5] .= simulate_game(U[s,n,:], 1, data2, game_parameters, simulation_parameters, cf, A2[1], A2[2])
            end

        end
    end

    return Results

end

loop_simulations (generic function with 1 method)

In [6]:
function simulate_game(U_sn, draw, data, game_parameters, simulation_parameters, cf, p_W = p_W, Ppt_array=Ppt_array, t=1, grid=100)


    #U_sn is a vector of U made for simulation s of observation n

    Vmax, L, Dj = data
    Ds = one(1.0) - Dj

    ρ, β, c0, λj = game_parameters;
    c1 = c0/30;
    
    
    #counterfactual_parameters####
    tsym = cf[1]
    ac = cf[2]
    acs = cf[3]
    acj = cf[4]



    hst = Int64(simulation_parameters[1])
    hjt = Int64(simulation_parameters[2])

    result = zeros(5);

    #number of observed proposals
    observed_proposals = 0.0


    #recovering "T+1" from s_W 
    T_game = size(p_W,1)
    
    t = Int64(1)


    while(result[3]==0.0 && t < T_game)


        #2 draws per turn
        u2 = U_sn[draw]
        draw+=1

        u3 = U_sn[draw]
        draw+=1


        #setting the default variables according to the propositor
        hst, hjt, hs_next, hj_next = choose_parameters(u2, u3, hst, hjt, Ppt_array)

        #proposal ####
        policy, payoff_prop = proposal(Ppt_array, t, hst, hjt)

        if(policy==3.0)

            observed_proposals += 1.0
            
            result[1] = payoff_prop
            result[2] = 0.0
            result[3] = 3.0
            result[4] = t
            result[5] = observed_proposals
            

        elseif(policy==2.0)

            t+=1

            hst, hjt = update_beliefs(hs_next, hj_next)

        else      
            #(policy==1.0)

            observed_proposals += 1.0
            
            
            result[1] = payoff_prop
            result[2] = 0.0
            result[3] = 1.0
            result[4] = t
            result[4] = observed_proposals
            

        if(t==T_game)

            result[1] = p_W[T_game, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
            result[2] = 0.0
            result[3] = 1.0
            result[4] = t
            result[5] = max(observed_proposals, 1.0)

        end
            
        end
        
        
    end
    


    return result
        
end


function choose_parameters(u2, u3, hst, hjt, Ppt_array)
    
    hs_next = draw_beta(u2, hst)
    hj_next = draw_beta(u3, hjt)

    return hst, hjt, hs_next, hj_next
end

choose_parameters (generic function with 1 method)

In [7]:
function draw_beta(u, hkt, β=game_parameters[2], grid=100)


    if(hkt == grid)

        return grid

    else

        x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
        x = floor(x * grid)
#         return Int64(x * 100) #to convert in an integer
        return Int64(x)

    end
end


#cost function
function Ct(t, c0=c0, c1=c1)

    #didn't use D because is normalized to D == 1.0
    #cost at period t=0(index1) is 0
    if(t <= 1)
        return 0
    else
        return c0 + c1 * (t-1) #test to make index==1 be t==0
    end
end



#liquidation payoffs
# function s_L(t, L, Ds, Dj, c0, c1, acs, acj)
#     return min(L - Ct(t, c0, c1) - acs, Ds)
# end


# function j_L(t, L, Ds, Dj, c0, c1, acs, acj)
#     return min(L - Ct(t, c0, c1) - s_L(t, L, Ds, Dj, c0, c1, acs, acj) - acj, Dj)
# end


function p_L(t, L, c0, c1, ac)
    
    return min(L - Ct(t, c0, c1) - ac)
end



function proposal(Ppt_array, t, hst, hjt)

    #ppt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação

    return policy, payoff_prop = Ppt_array[ t, hst, hjt, [end,end-1]]

end


#maximum value of reorganization each period
function Vt(Vmax, ρ, t, ac)

    if(t <=1)
        return Vmax
    else
        #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
        return ρ^(t-2) * Vmax - ac
    end


end


function update_beliefs(hs_next, hj_next)
    
    hst = hs_next
    hjt = hj_next

    return hst, hjt
end

update_beliefs (generic function with 1 method)

Inputs do jogo

In [8]:
S = 100
N = 66

pids = workers();
lengthworkers = Int64(length(pids)/2)
pool = pids[1:lengthworkers]


using DelimitedFiles
U = readdlm("U.csv", ',')
U = reshape(U, (S, N, 120));

#Vh/D, L/D, Dj/D
data1 = [0.86, 0.19, 0.8];
n1 = 49 #number of observations in cluster1

data2 = [2.65, 0.69, 0.74];
n2 = 17 #number of observations in cluster2

17

In [9]:
#parâmetros estimados
#melhor resultado entre o CSA e as iterações do NM até agora

game_parameters = [0.93, 7.856, 0.116, 0.99]; 

simulation_parameters = [35.0, 10.0, 2.387];

In [10]:
tsym = Int64(1)
ac = 0.0
acs = ac
acj = ac

cf = [tsym, ac, acs, acj]

4-element Array{Float64,1}:
 1.0
 0.0
 0.0
 0.0

In [11]:
aux = pids[2]

A2 = @spawnat aux solve_tree(data2, game_parameters, cf)
A1 = solve_tree(data1, game_parameters, cf)
A2 = fetch(A2);


simulated_moments = simulate_moments(U, S, N, game_parameters, simulation_parameters, cf, data1, data2, A1, A2, n1)

9-element Array{Float64,1}:
  2.164823154253474
  1.0
  2.164823154253474
  0.25757575757575757
  0.0
  0.0
  0.0
  1.27246923161885
 -2.800263561280985

In [12]:
#valor que o agia4 calcula para os momentos simulados
#writedlm com vírgula em julia faz o output ficar ruim quando há "ponto"
writedlm( "simulated_moments_centralplanner.csv",  simulated_moments, ',')

In [13]:
# U_sn = U[10,20,:]

# simulate_game(U_sn, 1, data1, game_parameters, simulation_parameters, cf, A1[1], A1[2], 1, 100)

In [17]:
data_moments = [1.8192, 0.8281, 2.7704, 0.9696, 0.5652, 0.5562, 0.7835, 0.4987, -0.0962];

In [15]:
moments = ["1. avg log number of months between observed proposals incourt",
    "2. fraction reorganized given that the case went into court",
    "3. ln duration of court cases in months",
    "4. fraction of cases incourt",
    "5. avg recovery rate for S given a REORGANIZATION in the 25% fastest cases",
    "6. avg recovery rate for J given a REORGANIZATION in the 25% fastest cases",
    "7. junior avg fraction gain given incourt REORGANIZATION",
    "8. total recovery rate given incourt REORGANIZATION",
    "9. slope of the regression of log(recovery rate | incourt reorg) ~ log(duration)"];

In [18]:
hcat(moments, simulated_moments, data_moments)

9×3 Array{Any,2}:
 "1. avg log number of months between observed proposals incourt"                    …   2.16482    1.8192
 "2. fraction reorganized given that the case went into court"                           1.0        0.8281
 "3. ln duration of court cases in months"                                               2.16482    2.7704
 "4. fraction of cases incourt"                                                          0.257576   0.9696
 "5. avg recovery rate for S given a REORGANIZATION in the 25% fastest cases"            0.0        0.5652
 "6. avg recovery rate for J given a REORGANIZATION in the 25% fastest cases"        …   0.0        0.5562
 "7. junior avg fraction gain given incourt REORGANIZATION"                              0.0        0.7835
 "8. total recovery rate given incourt REORGANIZATION"                                   1.27247    0.4987
 "9. slope of the regression of log(recovery rate | incourt reorg) ~ log(duration)"     -2.80026   -0.0962